In [1]:
import sys
sys.path.append('/scratch/ne2213/projects/tmp_packages')
# sys.path.append('/scratch/ne2213/projects/tmp_packages/')
sys.path.append('/scratch/ne2213/projects/LLM-AAC/LLM-AAC-main')

from transformers import AutoConfig
from mmpt.models.transformermodel import MMBertForJoint
import torch


In [2]:
# test_qformer.py

def main():
    # Dummy config object
    class DummyConfig:
        def __init__(self):
            self.dataset = type('dataset', (), {
                'bert_name': 'bert-base-uncased',
                'max_video_len': 32
            })()
            self.model = type('model', (), {
                'encoder_projector': 'qformer',
                'qformer_layers': 6,
                'query_len': 64,
                'use_seg_emb': False,
                'mm_encoder_cls': 'MMBertForJoint'
            })()

    config = DummyConfig()

    # Load config and instantiate model
    model_config = AutoConfig.from_pretrained(config.dataset.bert_name)
    model_config.max_video_len = config.dataset.max_video_len
    model_config.encoder_projector = config.model.encoder_projector
    model_config.qformer_layers = config.model.qformer_layers
    model_config.query_len = config.model.query_len
    model_config.use_seg_emb = config.model.use_seg_emb
    
    # Qformer ADD THESE:
    model_config.encoder_dim = 768
    model_config.llm_dim = 768


    model = MMBertForJoint(model_config)

    # Dummy input (batch_size=2, frames=8, dim=768)
    dummy_video = torch.randn(2, 8, 768)
    dummy_attn = torch.ones(2, 8, dtype=torch.long)

    # Forward Q-Former only
    print("Passing dummy video through Q-Former...")
    if model.use_qformer:
        out = model.projector(dummy_video, dummy_attn)
        print(f"Output shape: {out.shape}")


if __name__ == "__main__":
    main()

Q-Former projector initialized (EncoderProjectorQFormer).
Passing dummy video through Q-Former...
Output shape: torch.Size([2, 64, 768])
